In [6]:
import pandas as pd
import numpy as np

from scipy import stats
from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests 

**Question 2**

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:

AUCs.txt


Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [4]:
aucs  = pd.read_csv('AUCs.txt', sep = '\t', header = 0)
aucs.head()

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898


In [8]:
%%time
p_data = []

for i, lhs_column in enumerate(aucs.columns[1:]):
    for j, rhs_column in enumerate(aucs.columns[1:]):
        if i >= j:
            continue
        
        _, p = stats.wilcoxon(aucs[lhs_column], aucs[rhs_column])
        p_data.append([lhs_column, rhs_column, p])

CPU times: user 3.29 ms, sys: 3.07 ms, total: 6.36 ms
Wall time: 5.26 ms


In [10]:
auc_p = pd.DataFrame.from_records(p_data)
auc_p.columns = ['A', 'B', 'p']
auc_p

,A,B,p
0,C4.5,C4.5+m,0.010757
1,C4.5,C4.5+cf,0.861262
2,C4.5,C4.5+m+cf,0.015906
3,C4.5+m,C4.5+cf,0.046333
4,C4.5+m,C4.5+m+cf,0.327826
5,C4.5+cf,C4.5+m+cf,0.022909


**Question 5**

Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [13]:
reject, p_corrected, a1, a2 = multipletests(auc_p.p, 
                                            alpha = 0.05, 
                                            method = 'holm')

auc_p['p_corrected'] = p_corrected
auc_p['reject'] = reject
auc_p

,A,B,p,p_corrected,reject
0,C4.5,C4.5+m,0.010757,0.064543,False
1,C4.5,C4.5+cf,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,0.015906,0.079532,False
3,C4.5+m,C4.5+cf,0.046333,0.138998,False
4,C4.5+m,C4.5+m+cf,0.327826,0.655651,False
5,C4.5+cf,C4.5+m+cf,0.022909,0.091636,False


**Question 6**

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга

In [14]:
reject, p_corrected, a1, a2 = multipletests(auc_p.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh')
auc_p['p_corrected'] = p_corrected
auc_p['reject'] = reject
auc_p

,A,B,p,p_corrected,reject
0,C4.5,C4.5+m,0.010757,0.045818,True
1,C4.5,C4.5+cf,0.861262,0.861262,False
2,C4.5,C4.5+m+cf,0.015906,0.045818,True
3,C4.5+m,C4.5+cf,0.046333,0.069499,False
4,C4.5+m,C4.5+m+cf,0.327826,0.393391,False
5,C4.5+cf,C4.5+m+cf,0.022909,0.045818,True


Прим. Некорректно, так как выборки связанные